In [1]:
#### LIBRARIES NEEDED ####

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time

In [8]:
#### FUNCTIONS ####


## Get Influenza's related words

def get_outlinks(url):
    '''This function takes the out links of a page given in url.
        /wiki is the starting tag of wikipedia pages
        PMID is the tag for counting the citations in a page (we are not interested about it)'''
    
    base_wiki_url = "https://it.wikipedia.org/wiki/"
    
    # Request the page
    page_response = requests.get(url)
    
    # Parse the page
    soup = BeautifulSoup(page_response.content, "html.parser")
    
    #Add out-links
    edges = []
    one_hop_links = []
    
    # Get neighbors of seed-url
    for link in soup.find_all('a'):
        temp = str(link.get('href'))
        if temp[0:5] == "/wiki" and "PMID" not in temp and ":" not in temp and "secolo" not in temp:
            try:
                int(temp[-4:])
            except:
                one_hop_links.append(temp[6:])
                edges.append(frozenset(["influenza", temp[6:]]))
    
    one_hop_links = list(set(one_hop_links))
    
    print(one_hop_links)
        
    for page in one_hop_links:
        
        url = base_wiki_url + page
        
        # Request the page
        page_response = requests.get(url)
        
        # Parse the page
        soup = BeautifulSoup(page_response.content, "html.parser")
        
        # Get neighbors of seed-url
        for link in soup.find_all('a'):
            temp = str(link.get('href'))
            if temp[0:5] == "/wiki" and "PMID" not in temp and ":" not in temp and "secolo" not in temp:
                try:
                    int(temp[-4:])
                except:
                    edges.append(frozenset([page, temp[6:]]))
    
    return edges

def get_data(words):
    
    if len(words) > 10:
        return print("No man, you can put just 10 words maximum")

    bs_url = "https://tools.wmflabs.org/pageviews/?project=it.wikipedia.org&platform=all-access&agent=user&range=all-time&pages="+"|".join(words)

    driver = webdriver.Chrome()
    driver.get(bs_url)
    
    time.sleep(10)

    first_click = driver.find_elements_by_tag_name("button")
    first_click[6].click()
    
    driver.find_element_by_class_name("download-csv").click()




# Pipeline

- Get Influenza's related words
- Get Data and Ground Truth
- Build Time Series
- Measure Correlation
- Model it!

### Get Influenza's related words

Explain what we're going to do

In [9]:
# Starting Link
seed_url = "https://it.wikipedia.org/wiki/Influenza"

wiki_links = get_outlinks(seed_url)

['Sindrome_di_Reye', 'Legionellosi', 'Naso', 'EMedicine', 'Sarcoidosi', 'Testa', 'Influenza_spagnola', 'Salasso', 'Fotofobia', 'Otite', 'Bronchiolite_obliterante', 'Istituto_Superiore_di_Sanit%C3%A0', 'Pneumomediastino', 'Rinite_allergica', 'Rinite_vasomotoria', 'Vaccino_influenza_stagionale', 'Virus_parainfluenzali_umani', 'Digital_object_identifier', 'Aspergillosi_broncopolmonare_allergica', 'Insufficienza_respiratoria', 'Organizzazione_Mondiale_della_Sanit%C3%A0', 'Berilliosi', 'Setto_nasale#Ematoma_del_setto_nasale', 'Antivirali', 'Bronchiectasia', 'Sindrome_di_Mendelson', 'Malattia_polmonare_restrittiva', 'Tonsille', 'Wendell_Stanley', 'Organizzazione_mondiale_della_sanit%C3%A0', 'Influenzavirus_C', 'Frank_Macfarlane_Burnet', 'Laringospasmo', 'Edema_polmonare', 'Reazioni_di_ipersensibilit%C3%A0#Ipersensibilit.C3.A0_di_Tipo_I_-_Tipo_anafilattico', 'Malattia_da_reflusso_faringo-laringeo', 'Tosse', 'Panbronchiolite_diffusa', 'Rinite_medicamentosa', 'Epiglottide', 'Polmonite_interstiz

KeyboardInterrupt: 

In [34]:
a = [frozenset([4,5]),frozenset([4,5]),frozenset([5,6])]

In [35]:
set(a)

{frozenset({5, 6}), frozenset({4, 5})}